<a href="https://colab.research.google.com/github/yorkjong/vistock/blob/feature%2Fibd/notebooks/ibd_rs_rating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Stock Analysis and Ranking with IBD RS Rating, inspired by the Investor's Business Daily (IBD) methodology.

### Install and Setup

#### Install Packages

In [1]:
%pip install "git+https://github.com/yorkjong/vistock.git@feature/ibd"
%pip install requests-cache

  Cloning https://github.com/yorkjong/vistock.git (to revision feature/ibd) to /tmp/pip-req-build-04ngzwwo
  Running command git clone --filter=blob:none --quiet https://github.com/yorkjong/vistock.git /tmp/pip-req-build-04ngzwwo
  Running command git checkout -b feature/ibd --track origin/feature/ibd
  Switched to a new branch 'feature/ibd'
  Branch 'feature/ibd' set up to track remote branch 'feature/ibd' from 'origin'.
  Resolved https://github.com/yorkjong/vistock.git to commit 6f2e5e27bea3492d6ef1b274eba4c046c594ec6a
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 2.5 MB/s eta 0:00:00
  Created wheel for vistock: filename=vistock-0.5.0-py3-none-any.whl size=61845 sha256=fe872c03f05c831bb7def99ee90c756d75a75c242df5eb83d85bc77b762a3a73
  Stored in directory: /tmp/pip-ephem-wheel-cache-gbsgvi1_/wheels/07/97/7f/a72c7fbc975be0326ac45fec1ca0c872e2d6e2bbea96caf706
Successfully built vistock
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.4

#### Setup and Configuration

In [2]:
# @title Enable DataFrame Formatter
from google.colab import data_table
data_table.enable_dataframe_formatter()

In [3]:
# @title Enable Requests Cache
import requests_cache
requests_cache.install_cache('ibd_cache', expire_after=3600)

In [4]:
# @title Initialize Widgets
import ipywidgets as widgets
output = widgets.Output()

In [5]:
# @title GitHub
import base64
import requests
import pandas as pd
from io import StringIO


class GitHub:
    def __init__(self, repo_owner, repo_name, token, dir='', branch='main'):
        base = 'https://api.github.com/repos'
        dir = dir.strip('/')
        if dir:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents/{dir}'
        else:
            self.base_url = f'{base}/{repo_owner}/{repo_name}/contents'
        self.branch = branch
        self.token = token

    def _request(self, method, url, headers=None, params=None, json=None):
        response = requests.request(method, url, headers=headers,
                                    params=params, json=json)
        if response.status_code in [200, 201]:
            return response.json()
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def file_exists(self, file_path):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            return True
        elif response.status_code == 404:
            return False
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return None

    def list_filenames(self, dir_path=''):
        url = f'{self.base_url}/{dir_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }
        response = requests.get(url, headers=headers,
                                params={'ref': self.branch})
        if response.status_code == 200:
            files = response.json()
            return [item['name'] for item in files]
        else:
            print(f"Request failed: {response.status_code} - {response.json()}")
            return []

    def download_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Cannot download.")
            return None

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json',
        }

        file_info = self._request('GET', url, headers=headers)
        if file_info:
            download_url = file_info['download_url']
            response = requests.get(download_url)
            if response.status_code == 200:
                return StringIO(response.text)
            else:
                print(f"Failed to download file: "
                      f"{response.status_code} - {response.text}")
                return None
        return None

    def download_csv(self, file_path):
        file_content = self.download_file(file_path)
        if file_content:
            return pd.read_csv(file_content)
        else:
            return pd.DataFrame()

    def upload_file(self, file_path, content):
        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        encoded_content = base64.b64encode(content.encode()).decode()
        payload = {
            'message': 'Uploading file',
            'content': encoded_content,
            'branch': self.branch
        }
        self._request('PUT', url, headers=headers, json=payload)

    def upload_df_as_csv(self, file_path, df):
        """Upload a DataFrame to a CSV file."""
        if not file_path.endswith('.csv'):
            file_path += '.csv'
        csv_content = df.to_csv(index=False)
        self.upload_file(file_path, csv_content)

    def remove_file(self, file_path):
        if not self.file_exists(file_path):
            print(f"File '{file_path}' does not exist. Skipping deletion.")
            return

        url = f'{self.base_url}/{file_path}'
        headers = {
            'Authorization': f'token {self.token}',
            'Accept': 'application/vnd.github.v3+json'
        }

        # Fetch the file info to get the SHA needed for deletion
        file_info = self._request('GET', url, headers=headers)
        if file_info:
            sha = file_info['sha']
            payload = {
                'message': 'Deleting file',
                'sha': sha,
                'branch': self.branch
            }
            self._request('DELETE', url, headers=headers, json=payload)

#-------------------------------------------------------------------------------

from google.colab import userdata

github = GitHub(
    repo_owner='YorkJong',
    repo_name='stock-reports',
    token=userdata.get('GithubToken.stock-reports'),
    dir='ibd',
)

In [6]:
# @title Update and Filter DataFrame

def update_tickers_with_names(df, ticker_column, name_separator=','):
    """
    Update ticker codes in a DataFrame with their corresponding stock names.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    - name_separator: Separator used to join names (default is comma for multiple tickers).

    This function updates the specified column with the stock names instead of ticker codes.
    """
    # Iterate over the specified column in the DataFrame
    for index, row in df.iterrows():
        tickers = row[ticker_column].split(name_separator)  # Split the tickers string into a list
        stock_names = [tw.stock_name(ticker) for ticker in tickers]  # Get stock names for each ticker
        # Update the stock names back to the DataFrame
        df.at[index, ticker_column] = name_separator.join(stock_names)  # Join the names back into a string


def remove_ticker_suffix(df, ticker_column):
    """
    Remove the '.TW' or '.TWO' suffix from ticker codes in a DataFrame.

    Parameters:
    - df: The DataFrame containing ticker codes.
    - ticker_column: The name of the column containing ticker codes.
    """
    # Apply string replacement for each ticker in the specified column
    df[ticker_column] = df[ticker_column].str.replace('.TWO', '', regex=False)
    df[ticker_column] = df[ticker_column].str.replace('.TW', '', regex=False)


def filter_increasing_relative_strength(df):
    """
    Filter stocks with increasing Relative Strength over different time periods.

    This function filters the DataFrame to include only those stocks where:
    - Relative Strength is above 100.
    - Relative Strength has increased over the past 1 month, 3 months, and 6 months.
    Optionally, you can add a condition to check if Percentile is above 90.
    """
    return df[
        (df["Relative Strength"] > 100)
        & (df["Relative Strength"] > df["1 Month Ago"])
        & (df["1 Month Ago"] > df["3 Months Ago"])
        & (df["3 Months Ago"] > df["6 Months Ago"])
        # & (df["Percentile"] > 90)  # Uncomment to include Percentile filter
    ]


In [7]:
# @title Rank Function

import os
from datetime import datetime

from vistock import ibd
from vistock import tw
from vistock.stock_indices import get_tickers

def rank(code, period='2y', tickers_getter=get_tickers,
         ticker_ref='^GSPC', out_dir='out'):
    tickers = tickers_getter(code)

    output.clear_output()
    with output:
        rank_stock, rank_indust = ibd.rankings(tickers, period=period,
                                               ticker_ref=ticker_ref)
    if rank_stock.empty or rank_indust.empty:
        print("Not enough data to generate rankings.")
        return

    # Update the stock names back to the DataFrame
    #update_tickers_with_names(rank_stock, 'Ticker')
    #update_tickers_with_names(rank_indust, 'Tickers')

    # Remove the '.TW' or '.TWO' suffix
    remove_ticker_suffix(rank_stock, 'Ticker')
    remove_ticker_suffix(rank_indust, 'Tickers')

    # Save to CSV
    print("\n\n***")
    os.makedirs(out_dir, exist_ok=True)
    today = datetime.now().strftime('%Y%m%d')
    for df, kind in zip([rank_stock, rank_indust],
                           ['stocks', 'industries']):
        filename = f'{code}_{kind}_{period}_{today}.csv'
        github.upload_df_as_csv(filename, df)
        df.to_csv(os.path.join(out_dir, filename), index=False)
        print(f'Your "{filename}" is in the "{out_dir}" folder.')
    print("***\n")

    return rank_stock, rank_indust

### Glossary of Terms

source (The source of stocks to analyze):
- This could include stocks traded on exchanges or components of a specific index.
- Common abbreviation(s) for the exchange or market sector.  
  - For Taiwan Markets, possible values include:
    - `TWSE`: Taiwan Stock Exchange (台灣上市股票交易所）
    - `TPEX`: Taipei Exchange （上櫃交易所）
    - `ESB`: Emerging Stock Board （興櫃交易所）
  - Can also be combined with '+' (e.g., `TWSE+TPEX`, `TWSE+TPEX+ESB`)
  - For America Markets, possible values include:
    - `SPX`: S&P 500 (標普五百指數)
    - `DJIA`: Dow Jones Industrial Average (道瓊指數)
    - `NDX`: NASDAQ-100 (納斯達克一百指數)
    - `SOX`: PHLX Semiconductor Index （費半指數）
  - Multiple indices can be combined using '+' (e.g., `SPX+DJIA+NDX+SOX`)

period (Historical Data Time Range)：
- The time range for which to fetch historical data.
- `2y` means 2 years
- `6mo` means 6 monthes

RS (Relative Strength)
- Relative Strength (RS) is a metric used to evaluate the performance of a stock relative to a benchmark index.
  - A higher RS rating indicates that the stock has outperformed the index, while a lower RS rating suggests underperformance.
- The IBD RS calculates the performance of the last year, with the most recent quarter weighted double.

min_percentile (最小百分位)
- The minimum percentile for a stock to be included in the rankings.

### Error Messages

In [ ]:
display(output)

### RS Rating and Ranking

In [ ]:
source = "All Indices" #@param ["S&P 500", "Dow Jones Industrial Average", "NASDAQ 100", "PHLX Semiconductor", "All Indices"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    'S&P 500': 'SPX',
    'Dow Jones Industrial Average': 'DJIA',
    'NASDAQ 100': 'NDX',
    'PHLX Semiconductor': 'SOX',
    'All Indices': 'SPX+DJIA+NDX+SOX',
}

rank_stock, rank_indust = rank(code_from_name[source], period)
display(rank_stock)
display(rank_indust)

In [ ]:
# @title Top Percentile Stocks
min_percentile = 90 # @param {"type":"slider","min":1,"max":99,"step":1}
top_stocks = rank_stock[rank_stock[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
print(top_stock_list)

In [ ]:
# @title Filtered Stocks with Increasing RS > 100
filtered_rank_stock = filter_increasing_relative_strength(rank_stock)
display(filtered_rank_stock)

### RS Rating and Ranking for Taiwan Stocks

In [ ]:
from vistock import tw

source = "全部" #@param ["上市", "上櫃", "上市+上櫃", "興櫃", "全部"]
period = "2y" # @param ["6mo","1y","ytd","2y"]

code_from_name = {
    '上市': 'TWSE',
    '上櫃': 'TPEX',
    '上市+上櫃': 'TWSE+TPEX',
    '興櫃': 'ESB',
    '全部': 'TWSE+TPEX+ESB'
}

tw_stocks, tw_industries = rank(code_from_name[source], period,
     tickers_getter=tw.get_tickers, ticker_ref='^TWII')
display(tw_stocks)
display(tw_industries)



***
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "TWSE+TPEX+ESB_stocks_2y_20240818.csv" is in the "out" folder.
Request failed: 422 - {'message': 'Invalid request.\n\n"sha" wasn\'t supplied.', 'documentation_url': 'https://docs.github.com/rest/repos/contents#create-or-update-file-contents', 'status': '422'}
Your "TWSE+TPEX+ESB_industries_2y_20240818.csv" is in the "out" folder.
***



,Ticker,Sector,Industry,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
1685,8098,Technology,Semiconductor Equipment & Materials,695.62,838.67,762.40,1404.51,99,99,99,99,1
105,6144,Communication Services,Entertainment,440.94,353.05,127.34,84.52,99,99,91,20,2
633,8374,Industrials,Industrial Distribution,434.70,299.19,103.33,83.26,99,99,77,18,3
724,1799,Healthcare,Drug Manufacturers - Specialty & Generic,404.83,419.46,268.75,194.15,99,99,99,97,4
2037,6640,Technology,Semiconductors,383.26,319.59,245.60,196.13,99,99,99,97,5
...,...,...,...,...,...,...,...,...,...,...,...,...
637,6786,Technology,Semiconductors,50.31,40.23,39.34,75.29,0,0,0,6,2257
968,6620,Healthcare,Drug Manufacturers - Specialty & Generic,48.84,51.25,62.96,103.17,0,0,3,69,2258
825,6927,Technology,Electronic Components,47.44,50.92,57.05,86.49,0,0,1,28,2259
903,6617,Healthcare,Biotechnology,43.43,55.40,69.04,78.64,0,0,8,9,2260


,Industry,Sector,Relative Strength,1 Month Ago,3 Months Ago,6 Months Ago,Tickers,Percentile,1 Month Ago,3 Months Ago,6 Months Ago,Rank
72,Industrial Distribution,Industrials,172.08,139.65,89.78,89.48,"8374,911608,2373,3114",99,98,61,33,1
47,Real Estate—Development,Real Estate,160.91,145.17,126.14,112.97,"3188,1436,4907,6186,6212,6171,5206,2538",99,99,99,91,2
93,Utilities - Renewable,Utilities,140.02,159.15,102.45,109.14,"6869,6873,6806",98,99,87,84,3
67,Real Estate - Diversified,Real Estate,139.00,128.16,126.50,113.36,"1438,9946,2545,2520,2547",97,97,99,92,4
32,Real Estate - Development,Real Estate,137.96,125.43,119.34,106.87,"2524,5508,5455,2537,2718,6177,3489,2539,2548,5...",96,95,98,82,5
...,...,...,...,...,...,...,...,...,...,...,...,...
81,Auto Manufacturers,Consumer Cyclical,71.95,75.60,75.80,93.42,"2206,1599,2237,2204,2201,2227",3,9,10,48,103
104,Travel Services,Consumer Cyclical,71.13,85.39,97.56,87.21,"6961,2745,2731,5706,6242,2719,2743,2734",2,34,77,22,104
101,Pharmaceutical Retailers,Healthcare,70.34,66.86,70.39,86.56,"4173,4175,6469",1,0,2,20,105
65,Internet Retail,Consumer Cyclical,69.44,74.05,69.55,89.28,"8472,8454,2941,8477,2949,8044",0,4,1,33,106


In [ ]:
# @title Top Percentile Taiwan Stocks
min_percentile = 95 # @param {"type":"slider","min":1,"max":99,"step":1}

top_stocks = tw_stocks[tw_stocks[ibd.TITLE_PERCENTILE] >= min_percentile]
num_rows, _ = top_stocks.shape
print(f'\nnumber of filtered tickers: {num_rows}')
top_stock_list = list(top_stocks["Ticker"])
top_stock_list = [tw.stock_name(ticker) for ticker in top_stock_list]
print(top_stock_list)

In [ ]:
# @title Filtered Taiwan Stocks with Increasing RS > 100

def convert_to_list(value):
    if isinstance(value, str):
        return value.split(',')
    return value

filtered_tw_stocks = filter_increasing_relative_strength(tw_stocks)
update_tickers_with_names(filtered_tw_stocks, 'Ticker')
display(filtered_tw_stocks)

filtered_tw_industries = filter_increasing_relative_strength(tw_industries)
update_tickers_with_names(filtered_tw_industries, 'Tickers')
#filtered_tw_industries.loc[:, 'Tickers'] = filtered_tw_industries['Tickers'].apply(convert_to_list)
display(filtered_tw_industries)

### Remove files in GitHub Repository

In [ ]:
# @title CSV Deleter
import re
import ipywidgets as widgets

# Example filenames
with requests_cache.disabled():
    all_filenames = github.list_filenames()

# Function to extract unique dates from filenames
def extract_dates(filenames):
    date_pattern = r'\d{8}'
    dates = set()
    for fn in filenames:
        match = re.search(date_pattern, fn)
        if match:
            dates.add(match.group(0))
    return sorted(dates, reverse=True)  # Sort dates from newest to oldest

# Function to remove a file (replace with your actual implementation)
def remove_file(filename):
    print(f"Removing file: {filename}")
    with requests_cache.disabled():
        github.remove_file(filename)
    all_filenames.remove(filename)

#-------------------------------------------------------------------------------

# Update file selector options based on selected date
def update_file_selector(change):
    def selector_width(filenames):
        max_filename_length = max(len(fn) for fn in filenames)
        return f'{max_filename_length * 10}px'  # 10px width per character

    selected_date = change['new']
    lst_fns = [fn for fn in all_filenames if selected_date in fn]
    file_selector_widget.options = lst_fns
    file_selector_widget.rows = len(file_selector_widget.options)
    file_selector_widget.layout=widgets.Layout(width=selector_width(lst_fns))

# Function to delete selected files
def delete_files(button):
    selected_files = file_selector_widget.value
    for file in selected_files:
        remove_file(file)
    update_widgets()    # Update widgets after deletion

# Update widgets to reflect current state
def update_widgets():
    # Refresh the date selector
    dates = extract_dates(all_filenames)
    selected_date = date_selector_widget.value
    if selected_date not in dates:
        i = date_selector_widget.options.index(selected_date)
        if i > len(dates) - 1:
            selected_date = dates[-1]
        else:
            selected_date = dates[i]
    date_selector_widget.options = dates
    date_selector_widget.value = selected_date
    if not dates:
        file_selector_widget.options = []
        return
    update_file_selector({'new': date_selector_widget.value})

#-------------------------------------------------------------------------------

# Create a widget for selecting dates
def create_date_selector(dates):
    return widgets.Dropdown(
        options=dates,
        value = dates[0],
        description='Date:',
        disabled=False
    )

# Create a widget for selecting files
def create_file_selector(filenames):

    return widgets.SelectMultiple(
        options=[],
        value=[],
        description='Files',
        disabled=False,
    )

# Create widgets
dates = extract_dates(all_filenames)
date_selector_widget = create_date_selector(dates)
file_selector_widget = create_file_selector(all_filenames)
delete_button = widgets.Button(description="Delete Selected Files")
delete_button.on_click(delete_files)

# Initialize the file selector with the latest date
update_widgets()

# Set up the observer to update file selector when date is changed
date_selector_widget.observe(update_file_selector, names='value')

# Display widgets
display(date_selector_widget)
display(file_selector_widget)
display(delete_button)
